In [ ]:
import pandas as pd
import numpy as np
import re
#import tensorflow as tf
#from google.colab import drive    #google colab and drive linkage funtion


In [ ]:
!pip install datasets     #installing cos 1st public dataset gave me error for not having this.
!pip install transformers
!pip install torch

##1. Excel file data loading

In [ ]:
collected_data = pd.read_excel("/datacollection.xlsx")

In [ ]:
collected_data.shape[0]

In [ ]:
collected_data.head()

In [ ]:
collected_data.columns

In [ ]:
collected_data.dropna(axis=0)

In [ ]:
collected_data.shape[0]

In [ ]:
collected_data.dropna(axis=0, inplace=True)

In [ ]:
collected_data.shape[0]

In [ ]:
collected_data.columns

In [ ]:
collected_data_df = pd.DataFrame(collected_data)

In [ ]:
collected_data_df

In [ ]:
# collected_data_df ['value']= collected_data_df.iloc[:, 0]
# collected_data_df ['label'] = collected_data_df.iloc[:, -1]


In [ ]:
collected_data_df.shape[0]

In [ ]:
#collected_data_df.drop(['Incorrect Sentence', 'Corrected Sentence'],axis=1)
collected_data_df = collected_data_df.rename(columns={"Incorrect Sentence": "value", "Corrected Sentence": "label"})

In [ ]:
#collected_data_df.drop(['Incorrect Sentence', 'Corrected Sentence'],axis=1, inplace=True)

In [ ]:
collected_data_df

##Regular Epression

In [ ]:
#testing regular expression to remove () at tail parts.
sample = '5. after dinner we went for a walk. (missing comma after "dinner")'
output = re.sub(r'\(.*\)', r'',sample)    #removing the whole (). we use \( and \), unless we do, python will think it's expression.
output = re.sub(r'^[0-9.]+', r'',output)     # removing numbers at the beginning of the sentences and . (eg. 2.)
output = re.sub(r'^[a-z]', r'[A-Z]', output)
print (output)

In [ ]:
collected_data_df.tail(20)

In [ ]:
#removing the whole () from value column
collected_data_df['value'] = [re.sub(r'\(.*\)', r'',i) for i in collected_data_df['value']]

In [ ]:
collected_data_df.tail(20)

In [ ]:
#removing the whole () from label column
#collected_data_df['label'] = [re.sub(r'\(.*\)', r'',i) for i in collected_data_df['label']]
#closing this as we don't have () in label

##2. Public Dataset loading

In [ ]:
from datasets import load_dataset

public_dataset = load_dataset("MohamedAshraf701/lang-8")

In [ ]:
print (public_dataset)

In [ ]:
 public_df = pd.DataFrame(public_dataset['train'])

In [ ]:
public_df.shape[0]

In [ ]:
public_df.columns

In [ ]:
public_df.tail(10)

In [ ]:
#changing the name of column to the same as our own file. if we don't do that, when combined, there will be 4 columns.
public_df.rename(columns={'processed_input':'value', 'processed_output': 'label'},inplace=True)

In [ ]:
public_df.head(10)

In [ ]:
public_df.tail(20)

In [ ]:
public_df.dropna(axis=0, inplace = True)

In [ ]:
public_df.shape[0]

In [ ]:
#combining collected data and public data into one file.

combined_data = pd.concat([collected_data_df, public_df], axis=0)

In [ ]:
combined_data.shape[0]

In [ ]:
combined_data.tail(20)

In [ ]:
combined_data

In [ ]:
#removing extra white spaces
combined_data['value'] = [i.strip() for i in combined_data['value']]

In [ ]:
combined_data['label'] = [i.strip() for i in combined_data['label']]

In [ ]:
#replacing with 1 space for location with more than 1 spaces.
combined_data['value'] = [re.sub(r"\s+", " ", i) for i in combined_data['value']]

In [ ]:
combined_data['label'] = [re.sub(r"\s+", " ", i) for i in combined_data['label']]

In [ ]:
#capitalizing the first letter of the sentence
combined_data['value'] = [i.capitalize() for i in combined_data['value']]
combined_data['label'] = [i.capitalize() for i in combined_data['label']]

In [ ]:
combined_data.head(20)

In [ ]:
combined_data.tail(20)

##splitting the data for training and testing

In [ ]:
#training data as 70%
combined_data_size = int(combined_data.shape[0]*0.7)

#but which 70%? so assign first (head) 70% as training
train_combined_data = combined_data.head(combined_data_size)

test_combined_data = combined_data.tail(combined_data.shape[0] - combined_data_size)

In [ ]:
train_combined_data.shape[0]

In [ ]:
train_combined_data.head(10)

In [ ]:
print(train_combined_data.shape[0])
print(test_combined_data.shape[0])

In [ ]:
train_combined_data = train_combined_data.dropna(axis=0)

In [ ]:
test_combined_data = test_combined_data.dropna(axis=0)

In [ ]:
train_combined_data.shape[0]

In [ ]:
train_combined_data['value'] = 'grammar: ' + train_combined_data['value']
test_combined_data['value'] = 'grammar: ' + test_combined_data['value']

In [ ]:
train_combined_data.head(1)

##Modeling

In [ ]:
#loading tokenizer, T5Tokenizer: To tokenize the dataset which consists of grammatically incorrect and correct sentences.
from transformers import T5Tokenizer
from datasets import Dataset

In [ ]:
#https://debuggercafe.com/getting-started-with-grammar-correction/
model_name = 't5-small'
batch_size = 16   #Number of training samples processed at once (adjust based on GPU memory)
max_length = 128  #This is the maximum context length to consider for each sample in the JSON files. Beyond this length, the text samples will be truncated and smaller samples will be padded.
epochs = 3        #The number of epochs to train the model for.
num_workers = 2   #Number of CPU workers for data loading (lower this if using Colab/limited
out_dir = 'results_t5_small'  #This is the output directory to save intermediate results.

In [ ]:
tokenizer = T5Tokenizer.from_pretrained(model_name)

https://wandb.ai/mukilan/T5_transformer/reports/Exploring-Google-s-T5-Text-To-Text-Transformer-Model--VmlldzoyNjkzOTE2#2.-training:


https://docs.python.org/3/library/re.html#search-vs-match

##tokenizing the data

In [ ]:
# #kept getting error. and it says bcos i didn't convert to dataset. so had to convert pd to huggingface dataset
# train_huggingface_dataset = Dataset.from_pandas(train_combined_data)
# test_huggingface_dataset = Dataset.from_pandas(test_combined_data)

In [ ]:
# train_huggingface_dataset = train_huggingface_dataset.filter(lambda example: example["label"] is not None and example["label"].strip() != "")
# test_huggingface_dataset = test_huggingface_dataset.filter(lambda example: example["label"] is not None and example["label"].strip() != "")

In [ ]:
# train_combined_data.loc[:,'value'] = 'grammar: ' + train_combined_data['value']
# test_combined_data.loc[:,'value'] = 'grammar: ' + test_combined_data['value']

# #without .loc[:, I received below error
# # <ipython-input-42-0fdd2619f78b>:1: SettingWithCopyWarning:
# # A value is trying to be set on a copy of a slice from a DataFrame.
# # Try using .loc[row_indexer,col_indexer] = value instead

# # See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
# #   train_combined_data['value'] = 'grammar: ' + train_combined_data['value']


##tokenizing the dataset

In [ ]:
# # Function to convert text data into model value and targets
# def preprocess_function(examples):
#     inputs = [f"rectify: {inc}" for inc in examples['value']]
#     model_inputs = tokenizer(
#         inputs,
#         max_length=max_length,
#         truncation=True,
#         padding='max_length'
#     )
#     # Set up the tokenizer for targets
#     with tokenizer.as_target_tokenizer():
#         labels = tokenizer(
#             examples['label'],
#             max_length=max_length,
#             truncation=True,
#             padding='max_length'
#         )

#     #      # Convert padding tokens from 0 → -100 (so model ignores them)
#     # labels["input_ids"] = [
#     #     [(l if l != 0 else -100) for l in label] for label in labels["input_ids"]
#     # ]

#     # model_inputs["labels"] = labels["input_ids"]
#     # return model_inputs
#     # Ensure labels exist and are properly formatted


#     if "input_ids" in labels:
#         model_inputs["labels"] = labels["input_ids"]
#     else:
#         model_inputs["labels"] = [[-100] * max_length]  # Fill empty labels with ignored tokens

#     return model_inputs


In [ ]:
# # Apply the function to the whole dataset
# tokenized_train = train_huggingface_dataset.map(
#     preprocess_function,
#     batched=True,
#     remove_columns=["value", "label"]  # ✅ Prevents dataset conflicts
# )
#     # num_proc=8

In [ ]:
# tokenized_test = test_huggingface_dataset.map(
#     preprocess_function,
#     batched=True,
#     remove_columns=["value", "label"]  # ✅ Prevents dataset conflicts

#   #  num_proc=8
# )

In [ ]:
train_value = tokenizer(
        train_combined_data['value'].tolist(),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

test_value = tokenizer(
        test_combined_data['value'].tolist(),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
#https://huggingface.co/google-t5/t5-small
#https://debuggercafe.com/getting-started-with-grammar-correction/

train_label = tokenizer(
        train_combined_data['label'].tolist(),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )

test_label = tokenizer(
        test_combined_data['label'].tolist(),
        max_length=max_length,
        truncation=True,
        padding='max_length',
        return_tensors='pt'
    )
#https://stackoverflow.com/questions/75247437/valueerror-text-input-must-of-type-str-single-example
#https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
# Convert tokenized input & labels to Hugging Face Dataset format
train_dataset = Dataset.from_dict({
    "input_ids": train_value["input_ids"],
    "attention_mask": train_value["attention_mask"],
    "labels": train_label["input_ids"]
})

# Convert tokenized input & labels to Hugging Face Dataset format
test_dataset = Dataset.from_dict({
    "input_ids": test_value["input_ids"],
    "attention_mask": test_value["attention_mask"],
    "labels": test_label["input_ids"]
})
#https://huggingface.co/docs/datasets/en/use_datase

##loading model

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,Trainer, TrainingArguments #This is for the loading of the T5 model.

In [ ]:
# Load the pre-trained model
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Resize token embeddings (IMPORTANT: Add this here)
model.resize_token_embeddings(len(tokenizer))

In [ ]:
# Specify the device (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

In [ ]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir=out_dir,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=out_dir,
    eval_strategy='epoch',
    save_strategy='epoch',
    #save_steps=500,
    #eval_steps=500,
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to='tensorboard',
    dataloader_num_workers=num_workers,
    fp16=True,
)

In [ ]:
# Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Start training
history = trainer.train()

In [ ]:
# Save model & tokenizer after training
model.save_pretrained(out_dir)  # Use your defined output directory
tokenizer.save_pretrained(out_dir)

print(f"✅ Model and tokenizer saved to {out_dir}")